In [1]:

import numpy as np
from mazeN_class import *

actions = ['U', 'D', 'L', 'R'] # UP , DOWN , LEFT , RIGHT

def is_valid_move(x, y, n, maze):
    if 0 <= x < n and 0 <= y < n and maze[x][y] != 0:
        return True
    return False

def value_iteration(maze, gamma=0.9, epsilon=1e-6, max_k = 600):
    rows = cols = len(maze[0])
    
    start_cell = None
    goal_cell = None
    
    for i in range(rows):
        for j in range(cols):
            if maze[i][j] == 2:
                start_cell = (i, j)
            elif maze[i][j] == 3:
                goal_cell = (i, j)
    
    
    if start_cell is None or goal_cell is None:
        raise ValueError("Can't find both start and goal cells!")
    
    reward = np.zeros((rows, cols))
    # Intializing v0(s)
    V = np.zeros((rows, cols))
    
    k = 1
    while True:
        delta = 0
        k = k+1
        for i in range(rows):
            for j in range(cols):
                if maze[i][j] != 0:  # Ignore walls
                    max_v_iter = float('-inf')
                    for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                        nx, ny = i + dx, j + dy
                        if is_valid_move(nx, ny, rows, maze):
                            p = 1  
                            if maze[i][j] == 3: # Defining reward
                                reward = 11    
                            else:
                                reward = -1
                            v_iter = reward + gamma * p *(V[nx][ny])
                            max_v_iter = max(max_v_iter, v_iter)

                    delta = max(delta, abs(max_v_iter - V[i][j]))
                    V[i][j] = max_v_iter

        if delta < epsilon or k >= max_k:
            break
    
    policy = np.zeros((rows, cols), dtype=str)
    for i in range(rows):
        for j in range(cols):
            if maze[i][j] != 0:  # Ignore walls
                if maze[i][j] == 3:
                    policy[i][j] = 'S' # S -> stop
                    continue
                max_action = -1
                max_v_iter = float('-inf')
                for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                    nx, ny = i + dx, j + dy
                    if is_valid_move(nx, ny, rows, maze):
                        p = 1  
                        if maze[i][j] == 3:
                            reward = 10
                        else:
                            reward = -1
                        v_iter = p * (reward + gamma * V[nx][ny])
                        if v_iter > max_v_iter:
                            max_v_iter = v_iter
                            max_action = action
                policy[i][j] = actions[max_action]
            elif maze[i][j] == 0:
                policy[i][j] = 'X '
    
    return V, policy

def policy_modified(policy):
    n = len(policy[0])
    x = np.zeros((n,n), dtype=str)
    for i in range(n):
        for j in range(n):
            if policy[i][j] == 0:
                x[i][j] == 'U'
            elif policy[i][j] == 1:
                x[i][j] == 'D'
            elif policy[i][j] == 2:
                x[i][j] == 'L'
            elif policy[i][j] == 3:
                x[i][j] == 'R'
            elif policy[i][j] == 4:
                x[i][j] == 'X '
    return x

def draw_maze(canvas, row, col, color):
    x1 = col * CELL_SIZE
    y1 = row * CELL_SIZE
    x2 = x1 + CELL_SIZE
    y2 = y1 + CELL_SIZE
    canvas.create_rectangle(x1, y1, x2, y2, fill = color)

def visualize_values(values, maze):
    n = len(maze[0])
    
    root = Tk()
    root.title('Value function')
    canvas_side = n * CELL_SIZE
    canvas = Canvas(root, width = canvas_side, height = canvas_side, bg = 'grey')
    
    for i in range(n):
        for j in range(n):
            if maze[i][j] == 0:
                draw_maze(canvas, i, j, 'black')
            if maze[i][j] == 1:
                draw_maze(canvas, i, j, 'white')
            if maze[i][j] == 2:
                draw_maze(canvas, i, j, 'red')
            elif maze[i][j] == 3:
                draw_maze(canvas, i, j, 'green')
                
    for i in range(n):
        for j in range(n):
            x = j * CELL_SIZE + CELL_SIZE // 2
            y = i * CELL_SIZE + CELL_SIZE // 2
            canvas.create_text(x, y, text="{:.3f}".format(values[i][j]), font=('Arial', 10))
    
    canvas.pack()
    root.mainloop()

def visualize_policy(policy, maze):
    n = len(policy[0])
    
    up_arrow = '\u2191'  
    down_arrow = '\u2193'  
    left_arrow = '\u2190'  
    right_arrow = '\u2192'  
    
    root = Tk()
    root.title('Policy')
    canvas_side = n * CELL_SIZE
    canvas = Canvas(root, width = canvas_side, height = canvas_side, bg = 'grey')
    
    for i in range(n):
        for j in range(n):
            if maze[i][j] == 0:
                draw_maze(canvas, i, j, 'black')
            if maze[i][j] == 1:
                draw_maze(canvas, i, j, 'white')
            if maze[i][j] == 2:
                draw_maze(canvas, i, j, 'red')
            elif maze[i][j] == 3:
                draw_maze(canvas, i, j, 'green')
    
    for i in range(n):
        for j in range(n):
            if policy[i][j] == 'U':
                arrow_symbol = up_arrow
            elif policy[i][j] == 'D':
                arrow_symbol = down_arrow
            elif policy[i][j] == 'L':
                arrow_symbol = left_arrow
            elif policy[i][j] == 'R':
                arrow_symbol = right_arrow
            else:
                continue

            x = j * CELL_SIZE + CELL_SIZE // 2
            y = i * CELL_SIZE + CELL_SIZE // 2

            canvas.create_text(x, y, text=arrow_symbol, font=('Arial', 20))
            
    canvas.pack()
    root.mainloop()
            
def visualize_values(values, maze):
    n = len(maze[0])
    
    root = Tk()
    root.title('Value functions')
    canvas_side = n * CELL_SIZE
    canvas = Canvas(root, width = canvas_side, height = canvas_side, bg = 'grey')
    
    for i in range(n):
        for j in range(n):
            if maze[i][j] == 0:
                draw_maze(canvas, i, j, 'black')
            if maze[i][j] == 1:
                draw_maze(canvas, i, j, 'white')
            if maze[i][j] == 2:
                draw_maze(canvas, i, j, 'red')
            elif maze[i][j] == 3:
                draw_maze(canvas, i, j, 'green')
                
    for i in range(n):
        for j in range(n):
            x = j * CELL_SIZE + CELL_SIZE // 2
            y = i * CELL_SIZE + CELL_SIZE // 2
            canvas.create_text(x, y, text="{:.3f}".format(values[i][j]), font=('Arial', 14))
            
    canvas.pack()
    root.mainloop()

def solution_path(policy, start, goal):
    n = len(policy[0])
    
    current_row = start[0]
    current_col = start[1]
    print(current_row, current_col)
    path = []
    path.append((current_row, current_col))
    
    while True:
        if policy[current_row][current_col] == 'U':
            current_row = current_row - 1
        elif policy[current_row][current_col] == 'D':
            current_row = current_row + 1
        elif policy[current_row][current_col] == 'R':
            current_col = current_col + 1
        elif policy[current_row][current_col] == 'L':
            current_col = current_col - 1
        elif policy[current_row][current_col] == 'S':
            # path.append((current_row, current_col))
            break
        path.append((current_row, current_col))
    print(path)

n = 10
        
root = Tk()
root.title('Maze Generator')
canvas_side = n * CELL_SIZE
canva = Canvas(root, width = canvas_side, height = canvas_side, bg = 'grey')
canva.pack()

mazen = Mazen(n, canva)
maze, start, goal = mazen.generate_maze()    
maze[start[0]][start[1]] = 2
maze[goal[0]][goal[1]] = 3


print("Maze ------------")
for i in range(n):
    print(maze[i])
print("start: ", start)
print("goal: ", goal)

optimal_value_function_with_policy, optimal_policy_with_policy = value_iteration(maze)
# optimal_policy_with_policy = policy_modified(optimal_policy_with_policy)
print("Optimal Value Function:")
print(optimal_value_function_with_policy)
print("\nOptimal Policy:")
print(optimal_policy_with_policy)
visualize_policy(optimal_policy_with_policy, maze)
visualize_values(optimal_value_function_with_policy, maze)
solution_path(optimal_policy_with_policy, start, goal)
# mazen.visualize_policy(maze, optimal_policy_with_policy)
root.mainloop()


Maze ------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 0]
[0, 1, 1, 0, 0, 0, 2, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 1, 0]
[0, 1, 1, 1, 0, 0, 1, 0, 1, 0]
[0, 1, 0, 1, 1, 0, 1, 0, 1, 0]
[0, 1, 0, 0, 1, 1, 1, 0, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 3, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
start:  (2, 6)
goal:  (8, 4)
Optimal Value Function:
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.53295266  1.70328162  3.00364704  4.44849743
   6.05388668  7.83765245  9.81961436  0.        ]
 [ 0.         -1.46830915 -0.52034261  0.          0.          0.
   4.44849801  0.         12.0217942   0.        ]
 [ 0.          0.          0.          0.          0.          1.70328292
   3.00364821  0.         14.46866065  0.        ]
 [ 0.         20.20822186 17.18739967 14.4686597   0.          0.
   1.70328339  0.         17.18740111  0.        ]
 [ 0.         23.5

In [34]:
import numpy as np

maze = [[0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 1, 0, 1, 0],
[0, 1, 1, 0, 1, 1, 1, 0],
[0, 0, 1, 1, 1, 0, 3, 0],
[0, 1, 1, 0, 0, 0, 1, 0],
[0, 0, 1, 2, 1, 0, 1, 0],
[0, 1, 1, 0, 1, 1, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0]]

def get_valid_actions(state):
    actions = []
    row, col = state
    if row > 0 and maze[row-1][col] != 0:
        actions.append('U')
    if row < len(maze)-1 and maze[row+1][col] != 0:
        actions.append('D')
    if col > 0 and maze[row][col-1] != 0:
        actions.append('L')
    if col < len(maze[0])-1 and maze[row][col+1] != 0:
        actions.append('R')
    return actions

def policy_evaluation(policy, gamma=0.7, epsilon=0.00001):
    V = np.zeros_like(maze, dtype=np.float32)
    while True:
        delta = 0.0

        for row in range(len(maze)):
            for col in range(len(maze[0])):
                reward = -1
                if maze[row][col] == 0:
                    continue
                if maze[row][col] == 3:
                    reward = 10
                old_value = V[row][col]
                action = policy[row][col]
                if action == 'U' and row > 0:
                    new_value = reward + gamma * V[row-1][col]
                elif action == 'D' and row < len(maze)-1:
                    new_value = reward + gamma * V[row+1][col]
                elif action == 'L' and col > 0:
                    new_value = reward + gamma * V[row][col-1]
                elif action == 'R' and col < len(maze[0])-1:
                    new_value = reward + gamma * V[row][col+1]
                else:
                    new_value = 0.0  # Set to 0 for invalid actions
                V[row][col] = new_value
                delta = max(delta, abs(old_value - new_value))
        if delta < epsilon:
            break
    return V

def policy_improvement(V, gamma=0.7):
    policy = np.empty_like(maze, dtype=object)
    for row in range(len(maze)):
        for col in range(len(maze[0])):
            if maze[row][col] == 3:
                policy[row][col] = 'S'
                continue
            if maze[row][col] == 0:
                policy[row][col] = 'X'
                continue
            best_action = None
            best_value = float('-inf')
            actions = get_valid_actions((row, col))
            for action in actions:
                if action == 'U' and row > 0:
                    value = gamma * V[row-1][col]
                elif action == 'D' and row < len(maze)-1:
                    value = gamma * V[row+1][col]
                elif action == 'L' and col > 0:
                    value = gamma * V[row][col-1]
                elif action == 'R' and col < len(maze[0])-1:
                    value = gamma * V[row][col+1]
                if value > best_value:
                    best_value = value
                    best_action = action
            policy[row][col] = best_action
    return policy

def policy_iteration():
    policy = np.random.choice(['U', 'D', 'L', 'R'], size=(len(maze), len(maze[0])))
    i = 0
    while True:
        V = policy_evaluation(policy)
        new_policy = policy_improvement(V)
        if np.all(policy == new_policy): 
            break
        policy = new_policy
        i = i+1
    return policy, i

# Testing the policy_iteration function
policy, i = policy_iteration()
print("Iterations:", i)
print("Optimal Policy:")
print(policy)

Iterations: 7
Optimal Policy:
[['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
 ['X' 'D' 'X' 'X' 'D' 'X' 'D' 'X']
 ['X' 'R' 'D' 'X' 'R' 'R' 'D' 'X']
 ['X' 'X' 'R' 'R' 'U' 'X' 'S' 'X']
 ['X' 'R' 'U' 'X' 'X' 'X' 'U' 'X']
 ['X' 'X' 'U' 'R' 'D' 'X' 'U' 'X']
 ['X' 'R' 'U' 'X' 'R' 'R' 'U' 'X']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']]
